# Outage Duration Prediction

**Name(s)**: Neil Sharma, Xiang Ding

**Website Link**: (your website link)

## Code

In [52]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = 'plotly'

### Framing the Problem

Prediction Problem Type: This problem type is regression, as we are trying to predict a continuous quantity, in this case how long an outage occurs.

Response Variable: Our response variable will be, in minutes, how long an outage occurs for. This value can be seen in the dataset as 'OUTAGE.DURATION'

Metric: We will use mean squared error (MSE) loss as our metric for evaluation. We decided on MSE due to its sensitivity to outliers and its ability to be easily understood as it maintains the same units as our response variable, 'OUTAGE.DURATION'.

In [43]:
##################################################
#     DATA CLEANING CODE FROM PROJECT 3          #
##################################################
df = pd.read_excel('outage.xlsx', skiprows = 5)

df = df.set_index('OBS')
df = df.iloc[1: , :]
df = df[df.columns[1:]]

df['OUTAGE.START.DATE'] = pd.to_datetime(df['OUTAGE.START.DATE'])
df['OUTAGE.START.DATE'] = df['OUTAGE.START.DATE'].dt.date

df['OUTAGE.RESTORATION.DATE'] = pd.to_datetime(df['OUTAGE.RESTORATION.DATE'])
df['OUTAGE.RESTORATION.DATE'] = df['OUTAGE.RESTORATION.DATE'].dt.date
df['CUSTOMERS.AFFECTED_MISSING'] = df['CUSTOMERS.AFFECTED'].isna().astype(int)


df = df[["YEAR", "MONTH", 'NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY', 'CAUSE.CATEGORY', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED']]
df = df.dropna(subset=['ANOMALY.LEVEL', 'CLIMATE.CATEGORY', 'OUTAGE.DURATION'])
pd.set_option('display.max_columns', None)
df

,YEAR,MONTH,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED
OBS,,,,,,,,,,
1.0,2011.0,7.0,MRO,East North Central,-0.3,normal,severe weather,3060,NaN,70000.0
2.0,2014.0,5.0,MRO,East North Central,-0.1,normal,intentional attack,1,NaN,NaN
3.0,2010.0,10.0,MRO,East North Central,-1.5,cold,severe weather,3000,NaN,70000.0
4.0,2012.0,6.0,MRO,East North Central,-0.1,normal,severe weather,2550,NaN,68200.0
5.0,2015.0,7.0,MRO,East North Central,1.2,warm,severe weather,1740,250,250000.0
...,...,...,...,...,...,...,...,...,...,...
1527.0,2016.0,3.0,WECC,Northwest,1.6,warm,intentional attack,0,0,0.0
1529.0,2016.0,7.0,WECC,Northwest,-0.3,normal,system operability disruption,220,485,NaN
1530.0,2011.0,12.0,MRO,West North Central,-0.9,cold,public appeal,720,155,34500.0


### Baseline Model

In [51]:
X = df[['ANOMALY.LEVEL', 'CAUSE.CATEGORY']]
y = df['OUTAGE.DURATION']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Preprocessing for numerical data: no transformation needed
# Preprocessing for categorical data: OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', ['ANOMALY.LEVEL']),
        ('cat', OneHotEncoder(), ['CAUSE.CATEGORY'])
    ])

# Create a pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

# Train the model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 28734346.8315485


In [53]:
trace0 = go.Scatter(
    x=np.arange(len(y_test)),
    y=y_test,
    mode='markers',
    name='Actual Values'
)
trace1 = go.Scatter(
    x=np.arange(len(y_pred)),
    y=y_pred,
    mode='markers',
    name='Predicted Values'
)

# Create layout
layout = go.Layout(
    title='Actual vs Predicted Values',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Outage Duration')
)

# Create figure and add traces
fig = go.Figure(data=[trace0, trace1], layout=layout)

# Show plot
fig.show()

### Final Model

In [ ]:
# TODO

### Fairness Analysis

In [ ]:
# TODO